## NSO ZTP Service

We implemented a simple service in NSO called ztp which will allow us to make a mapping between a device serial number and the day0 configuration file which should be applied, as well as define additional information like hostname.
The service will also provide 2 actions:
- Provide the day0 configuration file content based on a serial number
- Onboard a device with a specified serial number – add the device to the device tree, fetch the SSH keys and do a sync-from. After the device is sync-ed a NETCONF notification will be created on the custom topic ztp.


[Yang model](../nso/packages/ztp/src/yang/ztp.yang)

[Service python file](../nso/packages/ztp/python/ztp/main.py)

[Template applied by the onboarding action](../nso/packages/ztp/templates/ztp-template.xml)


The NETCONF notification is sent on a custom notification stream called ztp whoich needs to be dfined in the NSO configuration file (ncs.conf)

```xml
      <stream>
        <name>ztp</name>
        <description>Notifications generated by the ZTP process</description>
        <replay-support>true</replay-support>
        <builtin-replay-store>
          <enabled>true</enabled>
          <dir>./state</dir>
          <max-size>S10M</max-size>
          <max-files>50</max-files>
        </builtin-replay-store>
      </stream>
```



### Configure the ztp service for the serial 36e6df2c

We associate the 36e6df2c serial (which is the serial of our virtual device) to the xr_script.py day0 config script. For exemplifying we provide 2 different day0 files - one python script and one bash script.

[xr_script.py](../nso/packages/ztp/day0_templates/xr_script.py)

[test.sh](../nso/packages/ztp/day0_templates/test.sh)

The get_day0 action will find the configured information saved for the specified serial and replace those parameters (hostname, serial, IP, ...) in the corresponding template before sending it in the response.


#### TASK
Open a new terminal (File -> New -> Terminal)

Open the nso cli:

[cisco@nso ncs-run]$ **ncs_cli -u admin -C**

admin connected from 127.0.0.1 using console on nso

admin@ncs# **config t**

Entering configuration mode terminal

admin@ncs(config)# **ztp 36e6df2c hostname devnet_lab_xr day0 xr_script.py ip 198.18.1.32**

admin@ncs(config-ztp-36e6df2c)# **commit**

Commit complete.

admin@ncs(config-ztp-36e6df2c)# **end**

admin@ncs# **show running-config ztp**

ztp 36e6df2c

 hostname devnet_lab_xr
 
 day0     xr_script.py
 
 ip       198.18.1.32

!
